In [2]:
from email.mime import base
from docx import Document
from docx2pdf import convert
from subprocess import Popen
from PyPDF2 import PdfFileReader as read
from pdf2image import convert_from_path
import numpy as np
import base64
import cv2
import os
from docx.enum.text import WD_COLOR_INDEX
from docx.document import Document as _Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
from docx.table import _Cell, Table
from docx.text.paragraph import Paragraph
import re
import docx
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import subprocess
from PyPDF2 import PdfFileReader


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def Size(filename):# tìm và chọn size của văn bản
    size = []
    doc = docx.Document(filename)
    for p in doc.paragraphs:
        for i in p.runs:
            if i.font.size != None:
                size.append(i.font.size/12700)

In [5]:
def iter_unique_cells(row): #(Hợp nhất cells theo dòng, bỏ qua các lần lặp lại)
    prior_tc = None
    for cell in row.cells:
        this_tc = cell._tc
        if this_tc is prior_tc:
            continue
        prior_tc = this_tc
        yield cell

In [9]:
def iter_block_items(parent):
    """
    Tạo tham chiếu đến từng đoạn và bảng con trong file doc, theo thứ tự tài liệu. 
    Mỗi giá trị trả về là một thể hiện của Bảng hoặc Đoạn văn. 
    'parent' thường là một tham chiếu đến một chính Đối tượng tài liệu, 
    hoạt động cho đối tượng _Cell | đoạn văn | bảng
    """
    if isinstance(parent, _Document):
        parent_elm = parent.element.body
        # print(parent_elm.xml)
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield Paragraph(child, parent)
        elif isinstance(child, CT_Tbl):
            yield Table(child, parent)

In [ ]:
def input_processing(input_file):
    ##    input: đường dẫn đến file docx cần xử lý
##    output: file đoc/docx, pdf, folder chứa images
##    docx - pdf - folder ảnh - chuyển ảnh thành data truyền mạng
    input_pdf, output_file = input_file_processing(input_file)
    input_image,number_page = pdf_to_img(input_pdf)
    img_org_base64 = []
    for file in range(number_page):
        image = input_image + '/' + str(file)
        img_org_base64.append(imageToBase64(image))
    return output_file,img_org_base64

In [ ]:
def deleteFileFolder(input_file_name):
    file = input_file_name+'.pdf'
    if os.path.exists(file):
        os.remove(file)
    folder_path = input_file_name+'_img'
    #print(folder_path)
    #if os.path.exists(folder_path):
        #shutil.rmtree(folder_path)

In [12]:
def replace_string(key, value,numberList, countKey,p):
    # print(key)
    # print(dict_key)
    """    
    split đoạn văn và key thành list
    kiểm tra xem key có xuất hiện trong đoạn không
    đếm số lần xuất hiện của key, nếu thứ tự nằm trong numberList thì thay đổi key
    thay đổi: xóa các từ khác, giữ lại từ đầu tiên của key, thay bằng value
    chuyển đoạn văn từ list về đoạn

    input: key, từ để đổi, danh sách vị trí đổi, số thứ tự key, đoạn văn chứa key
    output: đoạn văn đã được đổi từ ở vị trí chỉ định, số thứ tự key
    """
    line_split = p.text.split() # split đoạn
    key_split = key.split() # split key
    len_key = len(key_split)
    for i in range(len(line_split)):
        if re.findall(key_split[0],line_split[i],re.IGNORECASE):# nếu từ đầu trong key xuất hiện
            count = 0 # đếm từ trong key
            while count < len_key:
                if re.findall(key_split[count],line_split[i+count],re.IGNORECASE): ##so khớp không phân biệt hoa thường
                    count+=1 # đếm xem các từ trong key xuất hiện đủ chưa
                else:
                    break
            if count == len_key: # nếu đủ
                countKey += 1
                #punctuation =""
                #if re.match(r'\S', p.text): #so khớp với ký tự không phải chữ
                    #punctuation = line_split[i+count-1][-1] # dấu câu
                if countKey in numberList:  # bắt đầu thay đổi ở các vị trí cần thiết
                    count_1 = 1
                    while count_1 < len_key:
                        line_split[i+count_1] = "" #thêm u ở phía trước để xử lý ký tự tiếng việt nhá
                        count_1+=1
                    value = ''
                    line_split[i] = value #+punctuation
                    # run = p.add_run()
                    # font = run.font                    
                    for idx in range(len(p.runs)): #khởi tạo idx
                        if idx<len(line_split)-1: # so sánh giá trị của idx
                            p.runs[idx].text = line_split[idx] + ' '  #tạo đoạn thay đổi thêm vào
                        elif idx==len(line_split)-1:      #so sánh thay đổi
                            p.runs[idx].text = u" ".join(line_split[idx:])   #Thay đổi
                            p.runs[idx].text = u" ".join(p.runs[idx].text.split()) 
                        else:  # Xét định dạng của đoạn thay đổi
                            p.runs[idx].text = ''     
                        flag = check_font(p.runs[idx])
                        if flag['bold']:    
                            p.runs[idx].font.bold = True
                        if flag['italic']:
                            p.runs[idx].font.italic = True
                        if flag['underline']:
                            p.runs[idx].font.underline = True
                            
                    # print(value, countKey, key)
                    # run.text = u" ".join(line_split)
                    # a = run.text
                    # run.text = u" ".join(a.split()) #loai bỏ khoảng trắng trùng lặp   
                    # p.text = u" ".join(p.text.split())            
    return countKey

def check_font(para):
    flag = {
        'bold': 0,
        'italic':0,
        'underline':0,
    }
    if para.bold:
        flag['bold'] = 1
    if para.italic: 
        flag['italic'] = 1
    if para.underline: 
        flag['underline'] = 1
    return flag


def replace(filename,key,value,numberList,output_file):
    # print(dict_key)
    """
    hàm duyệt từng đoạn trong file
    tìm và thay thế từ ở vị trí chỉ định
    input: tên file, từ muốn đổi, từ để đổi, danh sách vị trí đổi
    output: file word đã được thay từ ở những vị trí chỉ định
    """
    countKey = 0 # khởi tạo số thứ tự key
    doc = Document(filename)
    par = doc.paragraphs[0]
    #Tạo Style của văn bản và  đưa dòng có key
    styles = doc.styles
    style = doc.styles['Normal']
    font = style.font
    if f"'{par.style.font.name}'" != None:
        font.name = f"'{par.style.font.name}'"
    sizes = Size(filename)
    run = 0
    for p in doc.paragraphs:
        for i in p.runs:
            if i.font.size != None:
                font.size = Pt(sizes[run])
                run += 1
    for block in iter_block_items(doc):
        if isinstance(block, Paragraph):
            if re.findall(key,block.text,re.IGNORECASE): #so khớp không phân biệt hoa thường
                countKey = replace_string(key,value,numberList,countKey,block)
        else:
            for row in block.rows:
                for cell in iter_unique_cells(row):
                    for p in cell.paragraphs:
                            if re.findall(key,p.text,re.IGNORECASE): #so khớp không phân biệt hoa thường
                                    #print(re.search(key,p.text,re.IGNORECASE))
                                    countKey = replace_string(key,value,numberList,countKey,p)                                
                                    #Đưa style vào từ chuyển đổi   
                                    styles = doc.styles
                                    style = doc.styles['Normal']
                                    font = style.font
                                    font.name = f"'{par.style.font.name}'" 
    doc.save(output_file)
path = r'C:\Users\User\Desktop\stemmer\static\uploads\332\PK\PK.docx'
replace(path,'dd','20',[2,3,4],'test.docx')
    

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def stage3(input_file,key,dict_key,count):
##    Bước thứ 3
##    Tìm kiếm và thay thế key bằng value ở những vị trí trong numberList
##    lưu lại file word
##    Đọc file word, chuyển pdf, chuyển sang ảnh
##    chuyển ảnh thành data truyền mạng
##
##    input: file word đầu vào, key, value, những vị trí cần thay thế
##    output: ảnh base64 để hiển thị
    input_file_name, input_file_end = os.path.splitext(input_file)
    output_file = input_file_name+input_file_end
    for counter in range (30):
        if os.path.exists(output_file):
            if input_file_name[-3:-1] == '_v':
                output_file = input_file_name[:-1]+str(counter)+input_file_end
            else:
                output_file = input_file_name+'_v'+str(counter)+input_file_end
        else:
            break
    if count != 1:
        output_file = input_file_name+input_file_end
    
    replace(input_file,key,dict_key,output_file)
    output_file,img_org_base64 = input_processing(output_file)
    # Xóa file pdf,folder chứa ảnh khi đã chuyển qua base64 xong
    deleteFileFolder(input_file_name)
    return img_org_base64,output_file
stage3(r'C:\Users\User\Desktop\stemmer\static\uploads\332\PK\PK.docx','dd',{'20': [2], '2022': [4]},[4])

In [ ]:
def input_file_processing(input_file):
    ##    input: file docx cần xử lý
##    output: file pdf/doc/docx, lưu trong folder chứa input_file
##    cần chuyển docx sang pdf
    input_file_name = os.path.splitext(input_file)[0]
    out_folder = input_file_name.split("/")[:-1]
    out_folder = "/".join(out_folder)
    # convert trên window
    #convert(input_file)
    
    # convert trên linux
    LIBRE_OFFICE = r"/usr/bin/lowriter"
    p = Popen([LIBRE_OFFICE, '--headless', '--convert-to', 'pdf', '--outdir',
               out_folder, input_file],shell = True)
    #print([LIBRE_OFFICE, '--convert-to', 'pdf', input_file])
    p.communicate()
    input_pdf = input_file_name + '.pdf'
    pdf = PdfFileReader(input_pdf)
    return input_pdf, input_file
def pdf_to_img(input_pdf):
    pdf = read(open(input_pdf,'rb'))
    #pdf = PdfFileReader(open(input_pdf,'rb'))
    number_page = pdf.getNumPages()
    folder = input_pdf.split('.pd')[0] + '_img'
    if not os.path.exists(folder):
       os.makedirs(folder)
    pages = convert_from_path(input_pdf, 200)
    counter = 0
    for page in pages:
        myfile = folder+'/'+str(counter)
        page.save(myfile, 'PNG')
        counter = counter+1
        if counter > number_page:
            break
    return folder,number_page
input_file_processing(r'C:\Users\User\Desktop\stemmer\static\uploads\332\PK\PK.docx')
# pdf_to_img('C:\\Users\\User\\Desktop\\stemmer\\static\\uploads\\332\\PK\\PK.pdf')

In [ ]:
def imageToBase64(image): 
##    imencode mã hóa định dạng img thành data truyền trực tuyến và gán nó vào bộ nhớ đệm
##    nén định dạng dữ liệu hình ảnh để tạo điều kiện cho việc truyền mạng
##    b64encode mã hóa hình ảnh thành nhị phân hiển thị bằng ASCII
##    decode chuyển dạng mã hóa về string
    image = cv2.imread(image)
    retval, buffer = cv2.imencode('.jpg', image)
    jpg_as_text = base64.b64encode(buffer)
    image_data = jpg_as_text.decode("utf-8")
    image_data = str(image_data)
    #print(image_data)
    image_data = 'data:image/png;base64,' + image_data
    return image_data

In [8]:
def replace_string(key, dict_key, countKey,p):
    # print(key)
    # print(dict_key)
    """    
    split đoạn văn và key thành list
    kiểm tra xem key có xuất hiện trong đoạn không
    đếm số lần xuất hiện của key, nếu thứ tự nằm trong numberList thì thay đổi key
    thay đổi: xóa các từ khác, giữ lại từ đầu tiên của key, thay bằng value
    chuyển đoạn văn từ list về đoạn

    input: key, từ để đổi, danh sách vị trí đổi, số thứ tự key, đoạn văn chứa key
    output: đoạn văn đã được đổi từ ở vị trí chỉ định, số thứ tự key
    """
    numberList = [ ]
    for v in dict_key:
        for numList in dict_key[v]:
            numberList.append(numList)
    line_split = p.text.split() # split đoạn
    key_split = key.split() # split key
    len_key = len(key_split)
    for i in range(len(line_split)):
        if re.findall(key_split[0],line_split[i],re.IGNORECASE):# nếu từ đầu trong key xuất hiện
            count = 0 # đếm từ trong key
            while count < len_key:
                if re.findall(key_split[count],line_split[i+count],re.IGNORECASE): ##so khớp không phân biệt hoa thường
                    count+=1 # đếm xem các từ trong key xuất hiện đủ chưa
                else:
                    break
            if count == len_key: # nếu đủ
                countKey += 1
                #punctuation =""
                #if re.match(r'\S', p.text): #so khớp với ký tự không phải chữ
                    #punctuation = line_split[i+count-1][-1] # dấu câu
                if countKey in numberList:  # bắt đầu thay đổi ở các vị trí cần thiết
                    count_1 = 1
                    while count_1 < len_key:
                        line_split[i+count_1] = "" #thêm u ở phía trước để xử lý ký tự tiếng việt nhá
                        count_1+=1
                    value = ''
                    
                    print(value, countKey, key)
                    for v in dict_key:
                        if countKey in dict_key[v]:
                            value = v
                            break
                    line_split[i] = value #+punctuation
                    # run = p.add_run()
                    # font = run.font                    
                    for idx in range(len(p.runs)): #khởi tạo idx
                        if idx<len(line_split)-1: # so sánh giá trị của idx
                            p.runs[idx].text = line_split[idx] + ' '  #tạo đoạn thay đổi thêm vào
                        elif idx==len(line_split)-1:      #so sánh thay đổi
                            p.runs[idx].text = u" ".join(line_split[idx:])   #Thay đổi
                            p.runs[idx].text = u" ".join(p.runs[idx].text.split()) 
                        else:  # Xét định dạng của đoạn thay đổi
                            p.runs[idx].text = ''     
                        flag = check_font(p.runs[idx])
                        if flag['bold']:    
                            p.runs[idx].font.bold = True
                        if flag['italic']:
                            p.runs[idx].font.italic = True
                        if flag['underline']:
                            p.runs[idx].font.underline = True
                            
                    # print(value, countKey, key)
                    # run.text = u" ".join(line_split)
                    # a = run.text
                    # run.text = u" ".join(a.split()) #loai bỏ khoảng trắng trùng lặp   
                    # p.text = u" ".join(p.text.split())            
    return countKey

def check_font(para):
    flag = {
        'bold': 0,
        'italic':0,
        'underline':0,
    }
    if para.bold:
        flag['bold'] = 1
    if para.italic: 
        flag['italic'] = 1
    if para.underline: 
        flag['underline'] = 1
    return flag


def replace(filename,key,dict_key,output_file):
    # print(dict_key)
    """
    hàm duyệt từng đoạn trong file
    tìm và thay thế từ ở vị trí chỉ định
    input: tên file, từ muốn đổi, từ để đổi, danh sách vị trí đổi
    output: file word đã được thay từ ở những vị trí chỉ định
    """
    countKey = 0 # khởi tạo số thứ tự key
    doc = Document(filename)
    par = doc.paragraphs[0]
    #Tạo Style của văn bản và  đưa dòng có key
    styles = doc.styles
    style = doc.styles['Normal']
    font = style.font
    if f"'{par.style.font.name}'" != None:
        font.name = f"'{par.style.font.name}'"
    sizes = Size(filename)
    run = 0
    for p in doc.paragraphs:
        for i in p.runs:
            if i.font.size != None:
                font.size = Pt(sizes[run])
                run += 1
    for block in iter_block_items(doc):
        if isinstance(block, Paragraph):
            if re.findall(key,block.text,re.IGNORECASE): #so khớp không phân biệt hoa thường
                countKey = replace_string(key,dict_key,countKey,block)
        else:
            for row in block.rows:
                for cell in iter_unique_cells(row):
                    for p in cell.paragraphs:
                            if re.findall(key,p.text,re.IGNORECASE): #so khớp không phân biệt hoa thường
                                    #print(re.search(key,p.text,re.IGNORECASE))
                                    countKey = replace_string(key,dict_key,countKey,p)                                
                                    #Đưa style vào từ chuyển đổi   
                                    styles = doc.styles
                                    style = doc.styles['Normal']
                                    font = style.font
                                    font.name = f"'{par.style.font.name}'" 
    doc.save(output_file)
path = r'C:\Users\User\Desktop\stemmer\static\uploads\332\PK\PK.docx'
replace(path,'dd',{{'20': [2], '2022': [4]},{'08': [3]}},'test.docx')

TypeError: unhashable type: 'dict'